# Basic Gates Kata

**Basic Gates** quantum kata is a series of exercises designed
to get you familiar with the basic quantum gates in Q#.
It covers the following topics:
* basic single-qubit and multi-qubit gates,
* adjoint and controlled gates,
* using gates to modify the state of a qubit.

Each task is wrapped in one operation preceded by the description of the task.
Your goal is to fill in the blank (marked with `// ...` comments)
with some Q# code that solves the task. To verify your answer, run the cell using Ctrl+Enter (⌘+Enter on macOS).

Most tasks in this kata can be done using exactly one gate.
None of the tasks require measurement, and the tests are written so as to fail if qubit state is measured.

The tasks are given in approximate order of increasing difficulty; harder ones are marked with asterisks.

## Part I. Single-Qubit Gates


### Theory

* A list of most common gates can be found in [this Wikipedia article](https://en.wikipedia.org/wiki/Quantum_logic_gate).
* [Quirk](http://algassert.com/quirk) is a convenient tool for visualizing the effect of gates on qubit states.

### Q# materials

* Basic gates provided in Q# belong to the `Microsoft.Quantum.Intrinsic` namespace and are listed [here](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.intrinsic).

> Note that all operations in this section have `is Adj+Ctl` in their signature.
This means that they should be implemented in a way that allows Q# 
to compute their adjoint and controlled variants automatically.
Since each task is solved using only intrinsic gates, you should not need to put any special effort in this.

### Task 1.1. State flip: $|0\rangle$ to $|1\rangle$ and vice versa

**Input:** A qubit in state $|\psi\rangle = \alpha |0\rangle + \beta |1\rangle$.

**Goal:**  Change the state of the qubit to $\alpha |1\rangle + \beta |0\rangle$.

**Example:**

If the qubit is in state $|0\rangle$, change its state to $|1\rangle$.

If the qubit is in state $|1\rangle$, change its state to $|0\rangle$.

> Note that this operation is self-adjoint: applying it for a second time
> returns the qubit to the original state. 

In [1]:
%kata T101_StateFlip

operation StateFlip (q : Qubit) : Unit is Adj+Ctl {
    //...
    // Hint use X(q);
}

The starting state:


|0⟩	0,6 + 0𝑖
|1⟩	0,8 + 0𝑖

The desired state:


|0⟩	0,8 + 0𝑖
|1⟩	0,6 + 0𝑖

The actual state:


|0⟩	0,8 + 0𝑖
|1⟩	0,6 + 0𝑖

Success!

### Task 1.2. Basis change: $|0\rangle$ to $|+\rangle$ and $|1\rangle$ to $|-\rangle$ (and vice versa)

**Input**: A qubit in state $|\psi\rangle = \alpha |0\rangle + \beta |1\rangle$.

**Goal**:  Change the state of the qubit as follows:
* If the qubit is in state $|0\rangle$, change its state to $|+\rangle = \frac{1}{\sqrt{2}} \big(|0\rangle + |1\rangle\big)$.
* If the qubit is in state $|1\rangle$, change its state to $|-\rangle = \frac{1}{\sqrt{2}} \big(|0\rangle - |1\rangle\big)$.
* If the qubit is in superposition, change its state according to the effect on basis vectors.

> Note:  
> $|+\rangle$ and $|-\rangle$ form a different basis for single-qubit states, called X basis.  
> $|0\rangle$ and $|1\rangle$ are called Z basis.


In [ ]:
%kata T102_BasisChange 

operation BasisChange (q : Qubit) : Unit is Adj+Ctl {
    // ...
}

### Task 1.3. Sign flip: $|+\rangle$  to $|-\rangle$  and vice versa.

**Input**: A qubit in state $|\psi\rangle = \alpha |0\rangle + \beta |1\rangle$.

**Goal** :  Change the qubit state to $\alpha |0\rangle - \beta |1\rangle$ (flip the sign of $|1\rangle$ component of the superposition).


In [ ]:
%kata T103_SignFlip 

operation SignFlip (q : Qubit) : Unit is Adj+Ctl {
    // ...
}

### Task 1.4 multiple single gates $|0\rangle$ to $|-\rangle$ and $|1\rangle$ to $|+\rangle$

In [ ]:
%kata T104_MultipleGates

operation MultipleGates(q : Qubit) : Unit is Adj+Ctl {
    // ...
}

# Quantum Random Number Generation Tutorial

True random number generation is a notoriously difficult problem. Many "random" generators today are actually pseudo-random, using a starting seed to spawning seemingly-random numbers that are actually a repeatable function of that seed. Most true random number generations are based on measurements of some natural phenomenon, such as atmospheric noise or atomic decay. 
(You can read more about it [here]( https://en.wikipedia.org/wiki/Random_number_generation).) 

Quantum random number generators (QRNGs) are truly random. The quantum algorithm for random number generation is one of the simplest applications of quantum computing principles, requiring very few qubits to run.

In this tutorial you will:
* learn about quantum random number generation and the principles behind it,
* implement a variety of QRNGs with equal probability of any given number,
* implement a single-bit QRNG with weighted probabilities of generated bits.

Let's go!

## Introduction

Recall from the [Qubit](../Qubit/Qubit.ipynb) tutorial that a qubit state $|\psi\rangle$ is defined via the basis states $|0\rangle$ and $|1\rangle$ as:

$$|\psi\rangle = \begin{bmatrix} \alpha \\ \beta \end{bmatrix} = \alpha|0\rangle + \beta|1\rangle\text{, where }|\alpha|^2 + |\beta|^2 = 1$$

We call $\alpha$ and $\beta$ the **amplitudes** of states $|0\rangle$ and $|1\rangle$, respectively. 
When $|\psi\rangle$ is measured in the $\{|0\rangle, |1\rangle\}$ basis (the computational basis), the probabilities of the outcomes are defined based on the state amplitudes: there is a $|\alpha|^2$ probability that the measurement result will be $0$, and a $|\beta|^2$ probability that the measurement result will be $1$.

> For example, a qubit in state $\begin{bmatrix} \frac{1}{\sqrt{2}} \\ \frac{1}{\sqrt{2}} \end{bmatrix}$ will yield measurement results $0$ or $1$ with equal probability, while a qubit in state $\begin{bmatrix} \frac{1}{2} \\ \frac{\sqrt3}{2} \end{bmatrix}$ will yield measurement result $0$ only 25% of the time, and $1$ 75% of the time.

This is sufficient to implement a simple random number generator!

> Remember that you can refer to the [Single Qubit Gates tutorial](../SingleQubitGates/SingleQubitGates.ipynb) if you need a refresher on the various quantum gates and their usage in Q#.

## <span style="color:blue">Exercise 1</span>: Generate a single random bit

**Input:** None.

**Goal:** Generate a $0$ or $1$ with equal probability.

<details>
    <summary><strong>Need a hint? Click here</strong></summary>
    Use the allocated qubit, apply a quantum gate to it, measure it and use the result to return a $0$ or $1$.
</details>

**Stretch goal:** Can you find a different way to implement this operation?

<details>
    <summary><strong>Need a hint? Click here</strong></summary>
    What are the different quantum states that produce $0$ and $1$ measurement results with the same probability? How would measuring the qubit in a different basis change the result? 
</details>


In [ ]:
%kata T1_RandomBit

operation RandomBit () : Int {
    using (q = Qubit()) {
        // ...
    }
}

## <span style="color:blue">Exercise 2</span>: Generate a random two-bit number

Now that you can generate a single random bit, you can use that logic to create random multi-bit numbers. Let's try first to make a two-bit number by combining two randomly generated bits.

**Input:** None.

**Goal:** Generate a random number in the range $[0, 2]$ with an equal probability of getting each of the four numbers.

**Stretch goal:** Can you do this without allocating qubits in this operation?

<details>
    <summary><strong>Need a hint? Click here</strong></summary>
    Remember that you can use the previously defined operations.
</details>

In [ ]:
%kata T2_RandomTwo

open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Convert;

operation RandomTwo () : Int {
    using (qs = Qubit[2]){

        return -1;
    }
}

## <span style="color:blue">Exercise 3</span>: Generate a random two-bit number

Now that you can generate a single random bit, you can use that logic to create random multi-bit numbers. Let's try first to make a two-bit number by combining two randomly generated bits.

**Input:** None.

**Goal:** Generate a random number in the range $[0, 3]$ with an equal probability of getting each of the four numbers.

**Stretch goal:** Can you do this without allocating qubits in this operation?

<details>
    <summary><strong>Need a hint? Click here</strong></summary>
    Remember that you can use the previously defined operations.
</details>

In [ ]:
%kata T3_RandomTwoBits

operation RandomTwoBits () : Int {
    // ...
    return -1;
}

## <span style="color:blue">Exercise 4</span>: Generate a number of arbitrary size

Let's take it a step further and generate an $N$-bit number. 

> Remember that you can use previously defined operations in your solution.

**Input:** An integer $N$ ($1 \le N \le 10$).

**Goal:** Generate a random number in the range $[0, 2^N - 1]$ with an equal probability of getting each of the numbers in this range.

> Useful Q# documentation: 
> * [`for` loops](https://docs.microsoft.com/quantum/user-guide/using-qsharp/control-flow#for-loop), 
> * [mutable variables](https://docs.microsoft.com/quantum/user-guide/using-qsharp/variables#mutable-variables), 
> * [exponents](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.math.powi).

In [ ]:
%kata T4_RandomNBits 

operation RandomNBits (N : Int) : Int {
    // ...
    return -1;
}